<a href="https://colab.research.google.com/github/deeplearningexplore/siamese/blob/main/question_pairs_detection1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c quora-question-pairs

Saving kaggle.json to kaggle.json
100% 4.95M/4.95M [00:00<00:00, 50.3MB/s]

 88% 153M/173M [00:01<00:00, 76.9MB/s]
100% 173M/173M [00:01<00:00, 97.2MB/s]
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
 43% 9.00M/21.2M [00:00<00:00, 13.2MB/s]
100% 21.2M/21.2M [00:00<00:00, 29.0MB/s]


In [214]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import pandas as pd
import tensorflow.keras.backend as K

%matplotlib inline

In [3]:
zip = zipfile.ZipFile("test.csv.zip", "r")
zip.extractall("./data")


In [4]:
zip = zipfile.ZipFile("train.csv.zip", "r")
zip.extractall("./data")

zip.close()

In [5]:
!rm "train.csv.zip"
!rm "test.csv.zip"

In [6]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")
print(train_data.head())
print(test_data.head())

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   id  qid1  ...                                          question2 is_duplicate
0   0     1  ...  What is the step by step guide to invest in sh...            0
1   1     3  ...  What would happen if the Indian government sto...            0
2   2     5  ...  How can Internet speed be increased by hacking...            0
3   3     7  ...  Find the remainder when [math]23^{24}[/math] i...            0
4   4     9  ...            Which fish would survive in salt water?            0

[5 rows x 6 columns]
  test_id  ...                                          question2
0       0  ...  Why did Microsoft choose core m3 and not core ...
1       1  ...        How much cost does hair transplant require?
2       2  ...                      What you send money to China?
3       3  ...                                  What foods fibre?
4       4  ...                     How their can I start reading?

[5 rows x 3 columns]


In [7]:
train_data.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [8]:
train_data = train_data[train_data["is_duplicate"]==1]
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [9]:
train_d1 = train_data['question1'].values
train_d2 = train_data['question2'].values

In [10]:
train_d1.shape

(149263,)

In [11]:
train_d = np.concatenate([train_d1,train_d2])

In [12]:
train_d.shape

(298526,)

In [13]:
oov_token = "<ukw>"
pad = "post"


In [14]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_d)
word_index = tokenizer.word_index


In [15]:
len(word_index)

31468

In [16]:
train_data1 = tokenizer.texts_to_sequences(train_d1)

In [17]:
train_data2 = tokenizer.texts_to_sequences(train_d2)

In [18]:
word_index["ambulance"]

19614

In [141]:
def data_generator(q1,q2,batch_size):
  indexes = list(range(len(q1)))
  indexes = tf.random.shuffle(indexes)
  length = len(q1)
  input1 = []
  input2 = []
  index = 0

  while True:
    if index==length:
      index = 0
      indexes = tf.random.shuffle(indexes)
    
    input1.append(q1[indexes[index]])
    input2.append(q2[indexes[index]])

    index+=1

    if len(input1)==batch_size:
      max_length = np.max((np.max([len(x) for x in input1]),np.max([len(x) for x in input2])))
      max_length = 2**int(np.ceil(np.log2(max_length)))

      input1 = tf.keras.preprocessing.sequence.pad_sequences(input1,maxlen=max_length,padding=pad,truncating=pad)
      input2 = tf.keras.preprocessing.sequence.pad_sequences(input2,maxlen=max_length,padding=pad,truncating=pad)

      yield [np.array(input1),np.array(input2)],np.zeros((2,batch_size, 10))

      input1 = []
      input2 = []



In [142]:
a,b=next(data_generator(train_data1,train_data2,5))

In [21]:
print(a[0])
print(b[0])

[[   17   305     6   354    19   102    20    33]
 [    3   349    10   579 16886     0     0     0]
 [    3     4    19  6769   505     0     0     0]
 [    3     4  1198     2 24782     0     0     0]
 [    5    98    15   250  8385   273   130     0]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [22]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"

--2020-10-14 08:22:22--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  48.9MB/s    in 16s     

2020-10-14 08:22:38 (41.9 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [23]:
zip = zipfile.ZipFile("wiki-news-300d-1M.vec.zip", "r")
zip.extractall("./data")
zip.close()

In [24]:
!rm "wiki-news-300d-1M.vec.zip"

In [25]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(tokens[1:], dtype='float32')
    return data

embedding_vectors =  load_vectors("data/wiki-news-300d-1M.vec")

In [26]:
len(embedding_vectors)

999994

In [27]:
vocab_size =  len(tokenizer.word_index) + 1
dimensions = 300
batch_size = 128

In [28]:
embedding_matrix = np.zeros((vocab_size, dimensions))
for word,index in word_index.items():
  val = embedding_vectors.get(word, None)
  if val is not None:
    embedding_matrix[index] = val

In [29]:
del embedding_vectors

In [131]:
# tf.compat.v1.enable_eager_execution()

In [157]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                            dimensions,
                                            weights=[embedding_matrix],
                                            trainable = False
                                            )

model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.LSTM(10))

input1 = tf.keras.Input(shape=(None,))
input2 = tf.keras.Input(shape=(None,))

vectors1 = model(input1)
vectors2 = model(input2)

output = tf.keras.layers.concatenate([vectors1, vectors2], axis=0)

saimese = tf.keras.Model(inputs = [input1, input2], outputs = output)

saimese.summary()


Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 10)           9453140     input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 10)           0           sequential_5[0][0]   

In [158]:
x = np.array(saimese([a,b]))
x.shape

(10, 10)

In [159]:
def cosine_similarity(a,b):
  return np.dot(a,b)/(np.linalg.norm(a) * np.linalg.norm(b))

In [286]:
def TripletLoss(batch_size, margin = 0.25):
  def lossfn(actual, vectors):
    anchor1 = vectors[:batch_size,:]
    anchor2 = vectors[batch_size:,:]

    anchor1 = anchor1 / K.expand_dims(K.sum(K.square(anchor1), axis=1), axis=-1)
    anchor2 = anchor2 / K.expand_dims(K.sum(K.square(anchor2), axis=1), axis =-1)
    similarity_matrix = K.dot(anchor1, K.transpose(anchor2))

    positive = K.sum(similarity_matrix * K.eye(batch_size), axis=1)
    temp1 = similarity_matrix - K.eye(batch_size) * 2.0
    temp2 = similarity_matrix * ( 1 - K.eye(batch_size))

    mean_negative = K.sum(temp2) / (batch_size-1)
    max_negative = K.max(temp1, axis=1)

    loss1 = K.maximum(0.0, mean_negative - positive + margin )
    loss2 = K.maximum(0.0, max_negative - positive + margin )

    loss = K.mean(loss1 + loss2)

    return loss
  return lossfn

In [287]:
saimese.compile(optimizer="adam", loss = TripletLoss(batch_size = batch_size))
tf.config.experimental_run_functions_eagerly(True)

In [288]:
saimese.fit_generator(data_generator(train_data1,train_data2,batch_size), epochs=10, steps_per_epoch=100)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/10
100/100 [==============================] - 8s 80ms/step - loss: 13.2004
Epoch 2/10
100/100 [==============================] - 8s 75ms/step - loss: 13.2002
Epoch 3/10
100/100 [==============================] - 8s 78ms/step - loss: 13.2001
Epoch 4/10
100/100 [==============================] - 8s 78ms/step - loss: 13.2001
Epoch 5/10
100/100 [==============================] - 8s 77ms/step - loss: 13.2001
Epoch 6/10
100/100 [==============================] - 8s 79ms/step - loss: 13.2001
Epoch 7/10
100/100 [==============================] - 8s 78ms/step - loss: 13.2000
Epoch 8/10
100/100 [==============================] - 8s 79ms/step - loss: 13.2000
Epoch 9/10
100/100 [==============================] - 8s 78ms/step - loss: 13.2000
Epoch 10/10
100/100 [==============================] - 8s 78ms/step - loss: 13.2000


In [59]:
def predict(a, b, margin=0):
  vectors = saimese([a,b])
  similarity = cosine_similarity(vectors[0], vectors[1])
  for i in similarity:
    if i > 0:
      print("similar")
    else: 
      print("Not Similar")
